<a href="https://colab.research.google.com/github/aliprg0/google_colab/blob/main/Several_Symbol_Model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install yfinance
import yfinance as yf   
import pandas as pd
import numpy as np

In [2]:
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense,LSTM
from tensorflow.keras.layers import Dropout
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.svm import SVC


In [29]:
#symbols = ["MSFT","AAPL","GOOG","TSLA","GGPI","BIM","AI","AMZN"]
#symbols = ["BTC-USD","LTC-USD","TRX-USD","XRP-USD","ETH-USD","BNB-USD","DASH-USD","VET-USD","LINK-USD","ADA-USD","DOT-USD","SOL-USD","BCH-USD","FTT-USD","FIL-USD","XMR-USD"]
#symbols = ["AAPL","MSFT","TSLA","GOOG"]
#symbols = ["BTC-USD","ETH-USD"]
symbols = ["BTC-USD"]
dfs = []
for i in symbols:
   data = yf.download(i,period="MAX",interval="1d")
   dfs.append(data)


[*********************100%***********************]  1 of 1 completed


In [30]:
fixed_dfs = []
for df in dfs:
    full = []
    df = np.array(df)
    for i in range(15, df.shape[0]-1):
        fifteen_days_ago = i-15
        fourteen_days_ago = i-14
        thirteen_days_ago = i-13
        twelve_days_ago = i-12
        eleven_days_ago = i-11
        ten_days_ago = i-10
        nine_days_ago = i-9
        eight_days_ago = i-8
        seven_days_ago = i-7
        six_days_ago = i-6
        five_days_ago = i-5
        four_days_ago = i-4
        three_days_ago = i-3
        two_days_ago = i - 2
        yesterday = i - 1
        today = i
        tomorrow = i + 1
        if df[tomorrow][3] > df[today][3]:
            future = 1
        else:
            future = 0


        new_lst = [df[fifteen_days_ago][3],df[fourteen_days_ago][3],df[thirteen_days_ago][3],df[twelve_days_ago][3],df[eleven_days_ago][3],
                   df[ten_days_ago][3], df[nine_days_ago][3], df[eight_days_ago][3], df[seven_days_ago][3], df[six_days_ago][3],
                   df[five_days_ago][3], df[four_days_ago][3], df[three_days_ago][3], df[two_days_ago][3], df[yesterday][3],
                   df[today][3], future]

        full.append(new_lst)


    fuu = np.array([full])
    lst = []
    for i in range(0, fuu.shape[1]):
        lst.append(fuu[0][i])
    
    mm = np.array(lst)


    column_names = ["fifteen_days_ago","fourteen_days_ago","thirteen_days_ago","twelve_days_ago","eleven_days_ago",
                      "ten_days_ago", "nine_days_ago", "eight_days_ago", "seven_days_ago", "six_days_ago",
                       "five_days_ago", "four_days_ago", "three_days_ago", "two_days_ago", "yesterday", 
                        "today", "result"]
    
    cnwr = ["fifteen_days_ago","fourteen_days_ago","thirteen_days_ago","twelve_days_ago","eleven_days_ago",
                      "ten_days_ago", "nine_days_ago", "eight_days_ago", "seven_days_ago", "six_days_ago",
                       "five_days_ago", "four_days_ago", "three_days_ago", "two_days_ago", "yesterday", 
                        "today"]
    dff = pd.DataFrame(mm, columns=column_names)
    
    scaler = MinMaxScaler()
    xx = dff["result"]
    dff = dff.drop(["result"],axis=1)
    scaled = pd.DataFrame(scaler.fit_transform(dff.T).T, columns=cnwr)

    scaled["result"] = xx
       
    fixed_dfs.append(scaled)



In [31]:
data = fixed_dfs[0]
for i in range(1,len(symbols)):
  data = pd.concat([data,fixed_dfs[i]], ignore_index = True)

In [32]:
data

,fifteen_days_ago,fourteen_days_ago,thirteen_days_ago,twelve_days_ago,eleven_days_ago,ten_days_ago,nine_days_ago,eight_days_ago,seven_days_ago,six_days_ago,five_days_ago,four_days_ago,three_days_ago,two_days_ago,yesterday,today,result
0,0.000000,0.018773,0.265120,0.229695,0.420227,0.486453,0.625681,0.746702,0.700186,0.766979,0.707251,0.581441,0.444757,0.436518,0.893931,1.000000,1.0
1,0.000000,0.242957,0.208019,0.395929,0.461244,0.598556,0.717911,0.672036,0.737909,0.679003,0.554925,0.420121,0.411997,0.863115,0.967724,1.000000,0.0
2,0.044114,0.000000,0.237266,0.319736,0.493114,0.643819,0.585894,0.669069,0.594691,0.438023,0.267812,0.257553,0.827161,0.959246,1.000000,0.930495,1.0
3,0.000000,0.202252,0.272551,0.420343,0.548808,0.499432,0.570332,0.506930,0.373382,0.228290,0.219545,0.705094,0.817687,0.852427,0.793179,1.000000,1.0
4,0.000000,0.064225,0.199245,0.316609,0.271499,0.336273,0.278350,0.156342,0.023788,0.015799,0.459389,0.562252,0.593990,0.539862,0.728811,1.000000,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5721,0.646123,0.621278,0.894155,1.000000,0.929963,0.842982,0.817145,0.793469,0.816981,0.751023,0.649888,0.467761,0.211682,0.223319,0.216449,0.000000,0.0
5722,0.635278,0.898068,1.000000,0.932552,0.848786,0.823905,0.801104,0.823747,0.760227,0.662831,0.487436,0.240824,0.252030,0.245414,0.036967,0.000000,1.0
5723,0.898068,1.000000,0.932552,0.848786,0.823905,0.801104,0.823747,0.760227,0.662831,0.487436,0.240824,0.252030,0.245414,0.036967,0.000000,0.063153,1.0
5724,1.000000,0.932552,0.848786,0.823905,0.801104,0.823747,0.760227,0.662831,0.487436,0.240824,0.252030,0.245414,0.036967,0.000000,0.063153,0.191157,1.0


In [33]:
data = data.dropna()

In [34]:
X = data.drop("result",axis=1)
xTrain, xTest, yTrain, yTest = train_test_split(X, data["result"], test_size=0.1,random_state=11)

print(xTrain.shape)
print(yTrain.shape)

print(xTest.shape)
print(yTest.shape)


(5153, 16)
(5153,)
(573, 16)
(573,)


In [35]:
clf = RandomForestClassifier(n_estimators=100)
clf.fit(xTrain,yTrain)
clf.score(xTest,yTest)

0.5113438045375218

In [36]:
logisticRegr = LogisticRegression()
logisticRegr.fit(xTrain, yTrain)
logisticRegr.score(xTest, yTest)

0.5270506108202443

In [37]:
from sklearn.neighbors import NearestCentroid
nc = NearestCentroid()
nc.fit(xTrain,yTrain)
nc.score(xTest,yTest)

0.5619546247818499

In [38]:
from sklearn.naive_bayes import GaussianNB
gnb = GaussianNB()
gnb.fit(xTrain, yTrain).score(xTest,yTest)

0.5584642233856894

In [39]:
from sklearn import tree
tree = tree.DecisionTreeClassifier()
tree.fit(xTrain, yTrain)
tree.score(xTest,yTest)

0.5130890052356021

In [40]:
from sklearn.ensemble import GradientBoostingClassifier
gb = GradientBoostingClassifier(n_estimators=100, learning_rate=1.0,
   max_depth=1, random_state=0).fit(xTrain, yTrain)
gb.score(xTest, yTest)

0.506108202443281

In [41]:
from sklearn.neighbors import KNeighborsClassifier
clf2 = KNeighborsClassifier(n_neighbors=7)
clf2.fit(xTrain, yTrain)
clf2.score(xTest,yTest)

0.5008726003490401

In [45]:
model = Sequential()
model.add(Dense(256, activation='relu', kernel_initializer='he_normal', input_shape=(16,)))
model.add(Dropout(0.2))
model.add(Dense(256, activation='relu', kernel_initializer='he_normal'))
model.add(Dropout(0.2))
model.add(Dense(256, activation='relu', kernel_initializer='he_normal'))
model.add(Dropout(0.2))
model.add(Dense(1, activation='sigmoid'))

model.compile(optimizer='adam', loss='binary_crossentropy',metrics=['accuracy'])

model.summary()

Model: "sequential_5"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_20 (Dense)            (None, 16)                272       
                                                                 
 dropout_15 (Dropout)        (None, 16)                0         
                                                                 
 dense_21 (Dense)            (None, 16)                272       
                                                                 
 dropout_16 (Dropout)        (None, 16)                0         
                                                                 
 dense_22 (Dense)            (None, 4)                 68        
                                                                 
 dropout_17 (Dropout)        (None, 4)                 0         
                                                                 
 dense_23 (Dense)            (None, 1)                

In [46]:
model.fit(xTrain,yTrain,epochs=30,batch_size=25,validation_data=(xTest,yTest))

Epoch 1/30
207/207 [==============================] - 1s 3ms/step - loss: 0.7995 - accuracy: 0.4898 - val_loss: 0.6919 - val_accuracy: 0.5445
Epoch 2/30
207/207 [==============================] - 1s 2ms/step - loss: 0.6975 - accuracy: 0.5038 - val_loss: 0.6916 - val_accuracy: 0.5271
Epoch 3/30
207/207 [==============================] - 1s 2ms/step - loss: 0.6953 - accuracy: 0.4991 - val_loss: 0.6922 - val_accuracy: 0.5148
Epoch 4/30
207/207 [==============================] - 1s 3ms/step - loss: 0.6929 - accuracy: 0.5077 - val_loss: 0.6920 - val_accuracy: 0.5166
Epoch 5/30
207/207 [==============================] - 0s 2ms/step - loss: 0.6928 - accuracy: 0.5145 - val_loss: 0.6923 - val_accuracy: 0.5236
Epoch 6/30
207/207 [==============================] - 1s 3ms/step - loss: 0.6930 - accuracy: 0.5127 - val_loss: 0.6931 - val_accuracy: 0.5113
Epoch 7/30
207/207 [==============================] - 1s 3ms/step - loss: 0.6924 - accuracy: 0.5191 - val_loss: 0.6924 - val_accuracy: 0.5253
Epoch 

In [44]:
svm = SVC()
svm.fit(xTrain,yTrain)
svm.score(xTest,yTest)

0.5445026178010471